# 네이버 카페 크롤러

네이버 카페 게시글을 검색 조건을 키워드로하여 크롤링 하는 코드 입니다.


[ 순서 ]
1. 라이브러리 import
2. 네이버 로그인
3. 검색 조건 지정하여 카페게시글 url 수집
4. url로 게시물 크롤링


## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import re #정규표현식
from selenium import webdriver #셀레니움 웹크롤러
from bs4 import BeautifulSoup as bs #웹크롤러
import requests #http 요청처리
import time
import warnings 
warnings.filterwarnings(action='ignore') #경고 메세지 안나오게 함

## 네이버 로그인

In [2]:
# 크롬 드라이버 제어
driver = webdriver.Chrome("C:\\Users\\mh\\jupyter code\\chromedriver.exe")

# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
time.sleep(2) 

# 아이디& 비밀번호 입력
my_id = '' # "id" 대신에 자신의 네이버 아이디가 들어가야 합니다
my_pw = '' # "password" 대신에 자신의 네이버 비밀번호가 들어가야 합니다

# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# '로그인' 버튼 클릭
driver.find_element_by_id('log.login').click()
time.sleep(1)

## 검색 조건 지정하여 카페게시글 url 수집
        

(참고)
- 개별 게시판 내에 검색어 지정하지 않는 경우 페이지 제한없으나
- 조건 검색의 경우 최대 페이지 100개로 제한돼있음 (개별 게시판에 들어가서 검색어 사용하는 경우에도 100개 제한)

In [3]:
from selenium.webdriver.common.keys import Keys

post_url_list = [] # 조건에 맞는 게시물의 url 수집할 빈 리스트

# 네이버 카페 접속(ex. 나이키매니아)
# ***로그인한 네이버 계정이 해당 카페에 접근 권한이 있어야 함***
time.sleep(2)
cafe_url = 'https://cafe.naver.com/parisienlook' # 크롤링하기 원하는 카페 url 넣어주기
driver.get(cafe_url) #chromedriver가 위에 입력한 url 주소로 이동함
time.sleep(1)


keyword = input('키워드 입력하세요 : ') # 직접 input 받을 키워드, 해당 카페에서 수집하길 원하는 키워드를 입력함
search = driver.find_element_by_id('topLayerQueryInput') 
search.send_keys(keyword)
search.send_keys(Keys.RETURN)
time.sleep(1)


# 프레임 전환(네이버 카페는 여러 프레임으로 구성되어서 크롤링할 게시글이 있는 프레임으로 전환해주어야 함)
driver.switch_to.frame('cafe_main') 
time.sleep(1)


# 노출 게시물 개수 50개로 설정
number_of_post = driver.find_element_by_id('listSizeSelectDiv') #노출 게시물 개수 버튼 
number_of_post.click()
aa = number_of_post.find_elements_by_tag_name('li') #5개씩, 10개씩, 15개씩, 20개씩 중에
aa[len(aa)-1].click() #50개씩 선택


# 검색 세부 조건 지정 0) 제외단어 지정하기 1)게시판 지정  2)기간 설정  
# 0) 제외단어 지정하기

driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click() #상세검색 버튼 클릭
time.sleep(1)

bye = '판매, 운포' #조건 설정중 다음단어제외 조건 설정에 들어갈 단어 #판매글 제외하기 위해 사용
#hi = '의류, 옷' #조건 설정중 다음 단어중 1개이상 포함 조건 설정에 들어갈 단어 

driver.find_element_by_xpath('//*[@id="srch_detail"]/div[2]/input').send_keys(bye) #다음단어제외 조건에 들어갈 단어 입력
#driver.find_element_by_xpath('//*[@id="srch_detail"]/div[3]/input').send_keys(hi) #1개이상 포험 조건에 들어갈 단어 입력
time.sleep(1)


# 1) 게시판 지정
menu = driver.find_element_by_id('divSearchMenuTop') #해당 상세검색 창에서 전체 게시판 버튼을 눌러서
menu.click() 
menu_name = menu.find_element_by_class_name('select_list').text # 게시판 리스트를 가져옴
for i,j in enumerate(menu_name.split('\n')):
    print(i,j) # 게시판 종류 라벨과 함께 출력 
    
menu_number = [] #수집하길 원하는 게시판을 리스트에 저장하기 위해 빈리스트 생성
while True:        
    #수집하길 원하는 게시판을 리스트에서 찾아서 기입함
    i = input('크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :') 
    if i != '끝': # 게시판 번호 입력 다했으면 끝 이라고 쓰면 멈춥니다
        menu_number.append(int(i))
    else:
        break

aaa = menu.find_elements_by_tag_name('li') #게시판 선택하기 위해 게시판 리스트 element 전부 aaa에 담아둠 위 크롤링할 게시판 번호랑 연결시킬 것임
                                        #len(aaa) 하면 해당 카페의 전체 게시판 갯수만큼 나옴

for i in menu_number: #위에서 지정한 게시판 리스트들로 for문을 반복함
    aaa[i].click() #게시판 하나를 가져옴
    print()
    print('-----',menu_name.split('\n')[i],'-----') #어떤 게시판에서 데이터가 수집하고 있는지 출력. menu_name은 게시판 리스트 텍스트
    
    # 2) 기간 설정
    months = ['01','02','03','04'] #검색 조건중 기간입력에 들어가는 월
    years = ['22'] #검색 조건중 기간입력에 들어가는 연도

    for year in years: #위에 지정한 연도에서 for문
        for month in months: #연도 안에서 월별로 for문
            
            driver.find_element_by_id('currentSearchDateTop').click() #기간입력 선택
            try:
                driver.find_element_by_id('input_1_top').click() #기간입력중 시작일자 지정
                driver.find_element_by_id('input_1_top').send_keys('20{0}{1}01'.format(year, month)) #시작일자
            except:
                break
                driver.implicitly_wait(30)
            try:
                driver.find_element_by_id('input_2_top').click() #기간입력중 종료일자 지정
                driver.find_element_by_id('input_2_top').send_keys('20{0}{1}30'.format(year, month)) #끝일자
            except:
                #종료일자가 0월 30일이 아니라 오늘 날짜가 돼야할 때 
                #끝일자를 오늘 날짜로 지정
                t = time.time()
                tmonth = int(time.strftime("%m", time.gmtime(t)))
                tdate = int(time.strftime("%d", time.gmtime(t)))-1
                driver.find_element_by_id('input_2_top').send_keys('20{}{}{}'.format(year,tmonth,tdate)) 
            driver.find_element_by_id('input_2_top').send_keys(Keys.RETURN) # 입력하고 엔터
            
            # 이제 연/월 하나의 쌍이 설정됨
            print('20{0}년 {1}월'.format(year, month)) #연/월 별로 데이터 수집되는 상황 출력
            
            driver.find_element_by_class_name('btn-search-green').click() # 조건 설정 완료 검색 버튼 클릭
            time.sleep(1)

            prev_next = driver.find_element_by_class_name('prev-next') # 아래 a가 들어있는 클래스
            buttons = prev_next.find_elements_by_tag_name('a') # 페이지 넘어가는 숫자 1~n

            page_num = 40 #수집하고 싶은 페이지 개수 지정 (게시물 개수 아님)
            
            if len(buttons) == 0: # 지정한 조건으로 게시물이 하나도 없을. 인식하는데 오래 걸림.
                pass
                
            else: #게시물이 있으면 url 수집함 
                try: # 중간중간 에러가 발생해서 전체를 try except안에 넣음
                    for p in range(1,page_num+1): #페이지마다 반복해서 수집 #1부터 page_num까지

                        # url 수집
                        soup = bs(driver.page_source, 'lxml') #p번째 페이지에 들어있는 게시글 50개(혹은 50개 미만)
                        post_url = soup.select('div .inner_list .article') #게시글 url이 inner_list > article에 들어있음        

                        # 카페 내 게시글의 url을 추출해서 만들어뒀던 post_url_list 리스트에 넣음
                        for i in post_url: #post_url엔 해당 페이지에 있는 게시글만큼 url이 들어있고
                            post_url_list.append('https://cafe.naver.com'+i['href']) #순서대로 저장함          
                        print(len(post_url_list)) #몇 개 수집됐는지 출력. #누적으로 나옴
                        driver.implicitly_wait(30)

                        prev_next = driver.find_element_by_class_name('prev-next') #아래 a가 들어있는 클래스
                        buttons = prev_next.find_elements_by_tag_name('a') # 페이지 넘어가는 숫자 1~n
                            
                            
                        # 페이지 이동 
                        # 페이지는 유형이 총 3가지가 있음
                        # 이전X 다음O 첫 페이지 / 이전O 다음O 중간 페이지 / 이전O 다음X 마지막 페이지
                        # 유형에 따라 buttons수가 달라짐
                        
                        if p<=10 & len(buttons)<12: #첫 페이지. #buttons의 수는 페이지수+다음버튼임 따라서 첫페이지라면 buttons수는 11(혹은 11미만)
                            if p%10 != 0: #10의 배수가 아니라면 
                                buttons[p].click() #p는 for문에서 1부터 시작하여 돌고있음. 여기서 button[0]은 1번째 페이지, button[1]은 2번째 페이지
                            else: #10의 배수라면 다음 버튼을 눌러줌
                                prev_next.find_element_by_css_selector('a.pgR').click()
                                
                        elif len(buttons) == 12: #이후 페이지 #다음 페이지라면 이전버튼+페이지수+다음버튼임 따라서 buttons수는 12
                            if p%10 != 0:
                                buttons[p%10+1].click() # p가 11이라면 button[2]를 누르게되는 것. 여기서 button[0]은 이전버튼, buttton[1]은 11페이지, button[2]는 12페이지 
                            else: 
                                prev_next.find_element_by_css_selector('a.pgR').click()

                        elif p>10 & len(buttons)<12 : #마지막 페이지 #이전버튼+페이지수임 따라서 buttons수는 11이하
                            buttons[p%10+1].click() 
                except: 
                    pass
                
    #해당 게시판에서 마지막 페이지까지 수집이 다 됐으면 다음 게시판으로 이동함
    menu = driver.find_element_by_id('divSearchMenuTop')
    menu.click() 
    aaa = menu.find_elements_by_tag_name('li')

키워드 입력하세요 : 골린이
0 전체 게시판
1 제휴 광고 문의
2 카페 고시 환율 1,380원 (06.13~06.19)
3 시크먼트 공지사항
4 시크먼트 이벤트
5 시크먼트 공구&핫딜
6 시크먼트 구매대행 견적문의
7 시크먼트 배송대행 견적문의
8 시크먼트 SNS
9 시크먼트 알쓸시잡
10 샤넬 팬 게시판
11 에르메스 팬 게시판
12 루이비통 팬 게시판
13 디올 팬 게시판
14 롤렉스 팬 게시판
15 반클리프아펠 팬 게시판
16 까르띠에 팬 게시판
17 셀린느 팬 게시판
18 고야드 팬 게시판
19 프리츠한센 팬 게시판
20 명품 OOTD
21 WEEKLY LUCKY DRAW
22 MIDNIGHT SHOCK DROP
23 CHIC앱 이용후기 이벤트
24 CHIC앱 가방
25 CHIC앱 지갑, 클러치
26 CHIC앱 신발
27 CHIC앱 의류
28 CHIC앱 시계, 액세서리
29 CHIC앱 빈티지
30 CHIC앱 Q&A
31 가입인사
32 등업 세레모니
33 ★회원나눔-꼬리잡기,퀴즈
34 스타벅스 프리퀸시 나눔
35 자유로운 토크
36 일상 토크
37 사회적 거리 두기
38 파인다이닝 & 미슐랭 토크
39 파인아트 토크
40 연애&사랑 토크
41 결혼 토크
42 반려동물 토크
43 여행 토크
44 골프 토크
45 유머 토크
46 인스타 맞팔, 블로그 서이추
47 명품 토크
48 명품 매장 재고, 오픈런 토크
49 명품 시계 토크
50 명품 골라주세요
51 명품 신상 사진
52 연예인 명품 패션
53 화장품 토크
54 향수 토크
55 보석 쥬얼리 토크
56 예물 토크
57 스드메 토크
58 자동차 토크
59 모피 토크
60 라식/라섹/노안 토크
61 성형 토크
62 피부 토크
63 치과/덴탈 클리닉 토크
64 다이어트 토크
65 명품 Q&A
66 자동차 Q&A
67 모피 Q&A
68 사이즈 Q&A
69 정품 확인 Q&A
70 명품착샷&소장품
71 명품 사용기, 리뷰
72 [동영상]명품 언박싱, 착샷, 소장품, 리뷰
73 명품 향수&화장품 리뷰
74 회원 거래

In [4]:
#수집한 url 확인
#set 함수로 중복 확인
#url은 안겹처도 전체 게시판과 다른 게시판에 중복되는 게시물이 있기 때문에 추후 중복 데이터 제거 필요
print(len(post_url_list), len(set(post_url_list)))
for i in post_url_list[:5]:
    print(i)

119 119
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=23451561&page=1&userDisplay=50&inCafeSearch=true&searchBy=0&query=%EA%B3%A8%EB%A6%B0%EC%9D%B4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%EC%9A%B4%ED%8F%AC&include=&exact=&searchdate=2022-01-012022-01-30&media=0&sortBy=date&articleid=3300493&referrerAllArticles=true
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=23451561&page=1&userDisplay=50&inCafeSearch=true&searchBy=0&query=%EA%B3%A8%EB%A6%B0%EC%9D%B4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%EC%9A%B4%ED%8F%AC&include=&exact=&searchdate=2022-01-012022-01-30&media=0&sortBy=date&articleid=3299828&referrerAllArticles=true
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=23451561&page=1&userDisplay=50&inCafeSearch=true&searchBy=0&query=%EA%B3%A8%EB%A6%B0%EC%9D%B4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%EC%9A%B4%ED%8F%AC&include=&exact=&searchdate=2022-01-012022-01-30&media=0&sortBy=date&articleid=3292120&referrerAllArticles=true
https://cafe.naver.com/ca-fe/ArticleRe

In [5]:
#수집한 url dataframe으로 저장
nm_url = pd.DataFrame(post_url_list) #리스트로 dataframe 만들기
nm_url.to_csv('1_nm_url_list.csv') #dataframe csv 형태로 저장하기 
nm_url.head() #만든 dataframe 출력해서 확인해보기

,0
0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
1,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
2,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
3,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
4,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...


## url로 게시물 크롤링

- 위에 1.0.1 라이브러리 import와 1.0.2 네이버 로그인이 실행된 상태에서 진행해야함 

In [6]:
post_url_list = pd.read_csv('1_nm_url_list.csv') #위에서 저장 url 파일 가져오기
a = post_url_list['0']

In [7]:
len(a)

119

In [8]:
# 제목, 본문, 댓글, 날짜 빈 list 생성
titles = []
reviews = []
comments = []
time_list = []

# 수집한 url 이 담겨있는 a에서 for문 반복
for url in a[:10]: #샘플로 슬라이싱 10으로 해둠**
        
    driver.get(url) #해당 url로 이동
    driver.implicitly_wait(10)

    # 프레임 전환 (여기까진 동적크롤러)
    try: 
        driver.switch_to.frame('cafe_main') 
        time.sleep(0.8)
        soup = bs(driver.page_source, 'lxml') 
    
    except UnexpectedAlertPresentException as e:
        pass
        
    
    # 데이터 수집은 정적크롤러
    # 제목 수집
    try:
        title = soup.find('h3',class_='title_text').text.strip() #제목 찾아서 수집
        titles.append(title)
    except: 
        try:
            title = driver.find_element_by_css_selector('h3.title_text').text.strip() #위 코드가 작동 안하는 경우가 있어서 같은 동작 다른 코드
            titles.append(title)
            time.sleep(0.8)
        except:
            title = [] #위 두개 코드로도 작동이 안되면 빈 리스트로 두고 다음 게시물로 넘어가도록 함
            titles.append(title)   
        
    # 본문 수집
    try:
        content = soup.select_one('div.se-module.se-module-text').text.strip() #본문 찾아서 수집
        reviews.append(content)
    except :
        try:
            content = soup.select_one('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우가 있어서 다른 코드
            reviews.append(content)
        except :
            try : 
                content = driver.find_element_by_css_selector('div.se-module.se-module-text').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                reviews.append(content)
            except :
                try :
                    content = driver.find_element_by_css_selector('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                    reviews.append(content)
                except :
                    content = [] #빈 리스트로 두고 다음 게시물로 넘어가도록 함
                    reviews.append(content)
                     
    # 댓글 수집
    iscomment = soup.find_all('span',class_='text_comment')
    if len(iscomment) != 0: # 댓글이 0개가 아니면 
        box = []
        for i in iscomment:
            box.append([i.get_text()])
    else: # 댓글이 0개면 넘어감
        box = []
        pass
    comments.append(box)
    
    #날짜 수집
    try:
        timeline = driver.find_element_by_xpath("/html/body/div/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]").text.strip()
        time_list.append(timeline)
    except:
        try:
            timeline = driver.find_element_by_css_selector("#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.date").text.strip()  
            time_list.append(timeline)
        except:
            timeline= []
            time_list.append(timeline)

In [9]:
#수집된 제목, 본문, 댓글 확인 
#3개 숫자가 맞아야 dataframe 만들 수 있음
print(len(titles))
print(len(reviews))
print(len(comments))
print(len(time_list))

#오류로 인해 안 맞으면 아래 코드로 맨뒤 게시글 하나 삭제해서 갯수 맞춰줌
#titles = titles[:-1] #타이틀이 본문이나 댓글보다 하나 많으면 슬라이싱 해줌
#len(titles)

10
10
10
10


In [10]:
#Dataframe 형태로 위에서 수집한 제목, 본문, 댓글을 합쳐줌
data = {'tit' : titles,
        'body' : reviews,
        'comment' : comments,
        'time' : time_list}

dataDF = pd.DataFrame(data)

In [11]:
#csv로 저장
dataDF['source'] = 'A' #해당 데이터 표시하기 위해 필요한 정보 추가 할 수 있음. 
dataDF['keyword'] = keyword #어떤 키워드로 데이터를 수집했는지 추가
dataDF.to_csv("1_nm_data.csv", encoding = "utf-8-sig") #한글 인코딩 utf-8-sig 
dataDF #저장된 모습 확인

,tit,body,comment,time,source,keyword
0,골프백 추천해주세용~~!!,바퀴있는걸로 사려구하는데 완전 골린이라 ... 뭘 사야할지 모르겠네요​,[[어뉴 바퀴달린거 이뻐요]],2022.01.30. 19:35,A,골린이
1,PXG 0211 어떤가요??,골프 입문 한달차 골린이예요 ㅎPXG 0211 풀셋 구입할까 고민중인데 어떤가요??,"[[pxg가 어려운채예요. 골린이라면 비추해요 일단 잘 맞아야 골프가 잼나죠~],...",2022.01.30. 13:26,A,골린이
2,제주 1박2일 2인 플레이 다녀왔어요.⛳,"안녕하세요, 시크님들❤골린이 생애 6번째 라운딩을 제주에서 하고 왔어요.멀리 한라산...",[[오~~~굿샷!♡ 친구랑 볼도치구 여행도하고 너무좋네요!저도 조만간 제주도 볼치러...,2022.01.27. 09:38,A,골린이
3,명x) 골린이 분들 계신가요?!,공.... 잘 맞으시나여? ㅠ전 뒷땅땜에 스트레스 받아 기절할거같아요 ㅋㅋㅋㅋ자세 ...,[[일주일차면 .. 잘맞는게 이상하죠!! 하시다보면 괜찮아지실거에용 전 5개월 넘엇...,2022.01.26. 14:47,A,골린이
4,(명x)골린이 골프백 제이린드버그랑 지포어 사용하시는 시크님들 어떠신가여 ?,이제 막 한달차 골린이예요 ! 주변에 골프치는 친구들이 없어서... 혼자서 돌아다녀...,[[저 지포어 써요 ㅎㅎㅎ 모델도 같네요. 전 골린이라 채 욕심이 없어서 만족하는데...,2022.01.26. 09:54,A,골린이
5,골프백 추천해주세요.,40대 진입한.. 골린이구요.골프 한달정도 배웠는데..골프채는 PXG꺼 사려고 정했...,[[파리게이츠나 마스터바니도 예뻐요 \n이번에 파리게이츠 모델 트와이스가 해요 ㅋㅋ...,2022.01.22. 00:06,A,골린이
6,퍼터 말이에용…,"퍼터를 구입해야 하는데재고도 요즘 별로 없고… 시타는 꿈도 못꿔보네요 ㅠㅠ​Pxg,...",[[골린이시면 오딧세이중에서 시타해보시고 고르시는게 나을듯해요\npxg가격대비 별로...,2022.01.21. 14:24,A,골린이
7,골린이 분들 계신가요ㅠㅠ 몸이 쑤셔요...,이제 3일차 골린이예요! 하프스윙 까지 배웠는데 팔도 아프고 여기저기 몸이 쑤시고 ...,[[이제 슬슬 주먹도 안쥐어지고 갈비뼈 등도 아프실거예요 ㅋㅋ손에 굳은살도 생기구여...,2022.01.19. 20:58,A,골린이
8,레슨 이제 시작,골린이예요ㅠㅎㅎㅎ프로님이 60분 꽉 채워서 옆에서 알려주시는데생각보다 자세가 너무 ...,[[골프는 자주 가서 레슨받는 만큼 빨리 느는 것 같아요! 테니스랑 같이 배웠었는데...,2022.01.18. 20:48,A,골린이
9,골프너무어려운운동이에요,배운지는 십년도넘었는데 필드도 여러번나갔지만아직도 아직도골린이에요^^ㅠㅠ,"[[ㅠㅠ], [골프는 늘 그래요 연습장 열심히 나가고 필드도 열심히 나가야 되요 ㅠ...",2022.01.16. 21:06,A,골린이
